🔥 Быстрая визуализация выбросов:

In [ ]:
import seaborn as sns
sns.boxplot(x=df['feature'])  # выбросы будут видны как точки

CLEANLAB

In [ ]:
!pip install cleanlab


In [ ]:
from cleanlab.classification import CleanLearning
from sklearn.ensemble import RandomForestClassifier
from cleanlab.dataset import cleanlab_dataset
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score
import pandas as pd

# Данные
X = df.drop("target", axis=1).values
y = df["target"].values

# 1. Предсказания через кросс-валидацию
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from cleanlab.classification import CleanLearning

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
clf = RandomForestClassifier()

# 2. CleanLearning wrapper
clean_model = CleanLearning(clf, seed=42, cv_n_folds=5)
clean_model.fit(X, y)

# 3. Какие метки подозрительные?
ranked_label_issues = clean_model.get_label_issues()
print(ranked_label_issues[:10])  # индексы строк, где вероятны ошибки


In [ ]:
from cleanlab.classification import CleanLearning
from sklearn.linear_model import LogisticRegression
import numpy as np

def auto_fix_with_cleanlab(X, y, strategy='fix'):
    """
    Автоматически исправляет метки на основе Cleanlab.

    strategy: 
        - 'fix'  → заменяет плохие метки на предсказанные
        - 'drop' → удаляет строки с ошибками
    """
    base_clf = LogisticRegression(max_iter=1000)
    clean_model = CleanLearning(base_clf)
    clean_model.fit(X, y)

    issues = clean_model.get_label_issues()
    print(f"[INFO] Найдено подозрительных меток: {len(issues)}")

    if strategy == 'fix':
        y_pred = clean_model.predict(X)
        y_fixed = y.copy()
        y_fixed[issues] = y_pred[issues]
        return X, y_fixed

    elif strategy == 'drop':
        X_clean = np.delete(X, issues, axis=0)
        y_clean = np.delete(y, issues, axis=0)
        return X_clean, y_clean

    else:
        raise ValueError("strategy must be 'fix' or 'drop'")


In [ ]:
from sklearn.linear_model import LogisticRegression
from cleanlab.classification import CleanLearning
from sklearn.metrics import accuracy_score
import numpy as np

def cleanlab_pipeline(X, y):
    """
    Полный pipeline:
    1. Обучает базовую модель
    2. Вычисляет accuracy до очистки
    3. Применяет Cleanlab для поиска ошибочных меток
    4. Заменяет подозрительные метки на предсказания
    5. Обучает новую модель и выводит accuracy после
    """
    # Шаг 1: базовая модель
    clf = LogisticRegression(max_iter=1000)
    clf.fit(X, y)
    preds = clf.predict(X)
    acc_before = accuracy_score(y, preds)

    # Шаг 2: Cleanlab
    clean_model = CleanLearning(LogisticRegression(max_iter=1000))
    clean_model.fit(X, y)
    issues = clean_model.get_label_issues()
    print(f"[INFO] Найдено подозрительных меток: {len(issues)}")

    # Шаг 3: исправим метки
    y_pred = clean_model.predict(X)
    y_fixed = y.copy()
    y_fixed[issues] = y_pred[issues]

    # Шаг 4: новая модель
    clf_fixed = LogisticRegression(max_iter=1000)
    clf_fixed.fit(X, y_fixed)
    preds_fixed = clf_fixed.predict(X)
    acc_after = accuracy_score(y_fixed, preds_fixed)

    # Вывод
    print(f"📊 Accuracy до очистки:  {acc_before:.4f}")
    print(f"✅ Accuracy после очистки: {acc_after:.4f}")
    return y_fixed, issues


Isolation forest

In [ ]:
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.datasets import load_breast_cancer
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import ace_tools as tools

# Загрузим пример табличных данных
data = load_breast_cancer()
df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = data.target

# Применим Isolation Forest для обнаружения выбросов
X = df.drop('target', axis=1)
iso = IsolationForest(contamination=0.05, random_state=42)
outlier_mask = iso.fit_predict(X) == 1  # 1 = норм, -1 = выброс

# Добавим колонку-флаг
df['is_outlier'] = ~outlier_mask  # True = выброс

# Кол-во выбросов
n_outliers = df['is_outlier'].sum()

# Отфильтруем очищенный датасет
df_clean = df[~df['is_outlier']].drop(columns='is_outlier')
tools.display_dataframe_to_user(name="Cleaned Dataset (No Outliers)", dataframe=df_clean)

# Вернём число выбросов
n_outliers


LOF

In [ ]:
from sklearn.neighbors import LocalOutlierFactor

# Применим LOF
lof = LocalOutlierFactor(n_neighbors=20, contamination=0.05)
lof_labels = lof.fit_predict(X)  # -1 = выброс
lof_mask = lof_labels == 1

# Добавим в PCA-датафрейм
df_pca['LOF_Outlier'] = ~lof_mask  # True = выброс

# Нарисуем сравнение
plt.figure(figsize=(8,6))
sns.scatterplot(data=df_pca, x='PC1', y='PC2', hue='LOF_Outlier', palette={False: 'blue', True: 'orange'})
plt.title("PCA Projection with LOF Outliers Highlighted")
plt.legend(title='LOF Outlier')
plt.grid(True)
plt.tight_layout()
plt.show()
